# QuartumSE Experiment Pipeline Notebook

This template mirrors the command-line pipeline for exploratory runs. Execute the cells sequentially or use *Run All* to reproduce the CLI output.


In [ ]:
from pathlib import Path

import yaml

from experiments.pipeline.metadata_schema import ExperimentMetadata

metadata_path = Path("experiments/shadows/examples/extended_ghz/experiment_metadata.yaml")
with metadata_path.open("r", encoding="utf-8") as fh:
    metadata_payload = yaml.safe_load(fh) or {}
metadata = ExperimentMetadata.model_validate(metadata_payload)
metadata


In [ ]:
import shutil

from experiments.pipeline.executor import execute_experiment

output_dir = Path("experiments/shadows/examples/extended_ghz/notebook_output")
if output_dir.exists():
    shutil.rmtree(output_dir)

execution_result = execute_experiment(metadata, output_dir=output_dir)
execution_result


In [ ]:
from experiments.pipeline.verifier import verify_experiment

verification = verify_experiment(execution_result["manifest_v1"])
verification


In [ ]:
import json

from experiments.pipeline.analyzer import analyze_experiment
from experiments.pipeline.run_full_pipeline import _normalise_targets

manifest_v0 = json.loads(execution_result["manifest_v0"].read_text(encoding="utf-8"))
manifest_v1 = json.loads(execution_result["manifest_v1"].read_text(encoding="utf-8"))
manifest_baseline = json.loads(execution_result["manifest_baseline"].read_text(encoding="utf-8"))

targets = _normalise_targets(metadata)
analysis = analyze_experiment(
    manifest_v0,
    manifest_v1,
    manifest_baseline,
    ground_truth=metadata.ground_truth,
    targets=targets if targets else None,
)
analysis


In [ ]:
from experiments.pipeline.reporter import generate_report
from experiments.pipeline.run_full_pipeline import _prepare_artifacts, _write_json

digest = execution_result["result_hash"].read_text(encoding="utf-8").strip()
analysis_path = output_dir / f"analysis_{digest}.json"
_write_json(analysis_path, dict(analysis))

artifacts = _prepare_artifacts(execution_result, digest)
report_path = output_dir / f"report_{digest}.html"
generate_report(metadata, artifacts, analysis, verification, report_path)

{
    "analysis_path": str(analysis_path),
    "report_path": str(report_path),
}


In [ ]:
from IPython.display import HTML

HTML(f"<a href='{report_path.as_posix()}' target='_blank'>Open generated report</a>")
